# ДЗ 1, часть 2. Теория информации

Имя, Фамилия: *напишите здесь*

группа: *напишите здесь группу, где вы числитесь*

**Оценка(для проверяющего):** 0 из 10

**Дедлайн:** одновременно с частью 3

In [ ]:
#all imports here please
import pickle as pkl

from collections import OrderedDict
import heapq

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#... add your own if necessary

Энтропию можно использовать не только для описания хаотичности, беспорядочности и уровня неопределённости, но и для того, чтобы исследовать зависимость между случайными величинами, используя теорию информации. Начнём с базовых вещей и постепенно дойдём до ультрапопулярной эвристики выбора важных признаков для машинного обучения -- взаимной информации.

## Задача 1. Поисследуем энтропию (2 балла)

Будем предполагать, что нам даны две случайных величины $X,Y$, можете предполагать, что они дискретные или имеют плотности, выкладок этот факт не изменит, но делайте всё по одному стандарту: укажите ниже и придерживайтесь его на всём протяжении ДЗ, если явно не просят сделать иначе.

**Choose your weapon/Выберите ваше оружие/Сhoisissez votre arme/Изаберите себи оружje:**  

$$
\int .. dx  ~~  \text{или} ~~ \sum_i  ~~\text{(выберите одно)}
$$  

### Независимость?

Покажите, что если две случайных величины $X,Y$ независимы, то $H(X,Y)=H(X)+H(Y)$.

Имейте в виду, что слева под знаком энтропии ДВЕ случайных величины, рассматриваемых как 2d-вектор, поэтому матожидание берётся по их совместному закону распределения. Это называется *совместной энтропией* величин $X,Y$.

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

### Зависимость?

Покажите, что в общем случае

$$
H(X,Y) = H(X) + H(Y \vert X)
$$
и вычислите, чему будет равен остаток $H(Y \vert X)$. Он носит название *условной энтропии*, не путайте с энтропией условного распределения.

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

!! Обоснуйте, какой может быть физический смысл $H(Y\vert X)$ (что она измеряет?) с точки зрения теории кодирования в случае дискретных СВ $X,Y$.

ВАШ КОММЕНТАРИЙ ТУТ

## Задача 2. Взаимная информация (2 балла)

Можно рассмотреть разницу

$$
I(X,Y) = H(X) - H(X \vert Y),
$$

которая отражает в некотором смысле (вы наверняка об этом думали в окошке выше) уровень использования информации о значении $Y$ при кодировании $X$ (если мы о кодировании). Эта величина носит название *взаимной информации* и является ещё одним хорошим способом исследования зависимости между случайными величинами. Библиотека `sklearn` позволяет оценивать её по данным и тем самым выделять самые важные признаки для моделей машинного обучения.

### Симметричность

Докажите, что $I(X,Y)=I(Y,X)$.

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

### Связь с KL

Докажите, что $I(X,Y)= D_{KL}( p_{X,Y} \vert p_{X}p_{Y})$. Слева совместное распределение $X,Y$, справа -- произведение вероятностей(или плотностей) $X,Y$. 

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

### Какой смысл?..

Как вы думаете, как интерпретируется $I(X,Y)$ в терминах теории кодирования (при дискретных $X,Y$)? Что измеряет эта величина?

ВАШ КОММЕНТАРИЙ ТУТ

## Максимальная энтропия (3 балла)

### Экспоненциальное распределение

(1 балл) Вычислите энтропию для экспоненциального распределения $Exp(\lambda)$ и распределения Лапласа $Laplace(0,\lambda)$, имеющее плотность $\frac{\lambda}{2} e^{-\lambda \vert x\vert }$.

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...


(2 балла) Докажите, что Экспоненциальное распределение $Exp(\lambda)$ имеет максимальную энтропию среди всех абсолютно непрерывных вероятностных распределений с фиксированным средним и имеющих ненулевую плотность в области $x\geq 0$.

ВАШЕ РЕШЕНИЕ И ОБОСНОВАНИЯ ТУТ...

## Строим свой код, основанный на частотах (3 балла)

Представим себе для примера достаточно большую книгу (например, Войну и Мир Льва Николаевича Толстого). Для того, чтобы записывать буквы в кодировке UTF-8 требуется от 2 до 4 байт (каждый 8бит). Но книга -- это не просто какой-то iid равновероятный поток символов, а структурированный текст, структуру которого можно по-разному использовать. К примеру, мы можем задуматься над тем, чтобы сжать её, используя информацию о том, какие символы встречаются чаще: ведь более частые можно кодировать более короткими кодовыми словами, а более редкие -- более длинными.

В этом идея [кода Хаффмана](https://compression.ru/download/articles/huff/huffman_1952_minimum-redundancy-codes.pdf), который мы попробуем поисследовать в самой простой наивной имплементации.

### Вычисляем языковую статистику

Возьмём для примера текст вики-страницы [МО1](http://wiki.cs.hse.ru/%D0%9C%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_1)

In [ ]:
with open("moText.txt","r", encoding="utf-8") as f:
    MOText = "\n".join(f.readlines())

print(MOText[:180])

Это не сильно большой, но не очень маленький текст.

In [ ]:
print(len(MOText))

Вычислите частоту каждого символа, сохранив их в упорядоченный словарь (OrderedDict, пригодится для сортировок)  вида {<символ>: <частота>}.

In [ ]:
#compute OrderedDict
#symFreqs = #YOUR CODE

In [ ]:
for key,val in symFreqs.items():
    print(f"{key}: {val}")
print(f"In total {len(symFreqs)} unique symbols, max freq= {max([val for val in symFreqs.values()])}")

### Строим Кодер и Декодер Хаффмана

Это хорошее упражнение для ценителей алгоритмов, но давайте сократим себе время и попробуем адаптировать одну из [доступных реализаций](https://www.geeksforgeeks.org/huffman-coding-in-python/) под наш сценарий, обернув в привычную классовую структуру и сохранив максимальную представленность самого алгоритма.

In [ ]:
#taken from https://www.geeksforgeeks.org/huffman-coding-in-python/
class Node:
    def __init__(self, symbol=None, frequency=None):
        #print("CREATING", symbol, frequency)
        self.symbol = symbol
        self.frequency = frequency
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequency < other.frequency

def build_huffman_tree(chars, freq):
  
    # Create a priority queue of nodes
    priority_queue = [Node(char, f) for char, f in zip(chars, freq)]
    heapq.heapify(priority_queue)

    # Build the Huffman tree
    while len(priority_queue) > 1:
        left_child = heapq.heappop(priority_queue)
        right_child = heapq.heappop(priority_queue)
        merged_node = Node(frequency=left_child.frequency + right_child.frequency)
        merged_node.left = left_child
        merged_node.right = right_child
        heapq.heappush(priority_queue, merged_node)

    return priority_queue[0]

def generate_huffman_codes(node, code="", huffman_codes={}):
    if node is not None:
        if node.symbol is not None:
            huffman_codes[node.symbol] = code
        else:
            generate_huffman_codes(node.left, code + "0", huffman_codes)
            generate_huffman_codes(node.right, code + "1", huffman_codes)

    return huffman_codes

def runTest():
    # Given example (try it to test)
    chars = ['a', 'b', 'c', 'd', 'e', 'f']
    freq = [4, 7, 15, 17, 22, 42]

    # Build the Huffman tree
    root = build_huffman_tree(chars, freq)

    # Generate Huffman codes
    huffman_codes = generate_huffman_codes(root)

    # Print Huffman codes
    for char, code in huffman_codes.items():
        print(f"Character: {char}, Code: {code}")
        
runTest()

Надеемся, что этот код заработал.... Теперь напишем кодер, используя эту технологию.

In [ ]:
class HuffmanCoderDecoder:
    
    def __init__(self,symbolProbaTable):
        """_summary_

        Args:
            symbolProbaTable (dict): a dict with symbol(keys) and probabilities (val)
        """ 
        self.symbols = list(symbolProbaTable.keys())
        self.probas = list(symbolProbaTable.values())
        self.symbolProbaTable = symbolProbaTable
        self.root = None
        self.fit()
        
    def fit(self):
        """
        Sets up encoder and decoder tables
        """        
        #YOUR CODE....
        #self.encoderTable = YOUR CODE (dict)
        #self.decoderTable = YOUR CODE (dict)
        pass
        
    def encode(self, text):
        """Encodes the text and returns encoded text

        Args:
            text (str): text to encode
        Returns:
            encodedText (str): encoded text
        """
        #YOUR CODE....
        pass        
        #return ....
    
    def decode(self, text):
        """Decodes the encoded text and returns decoded text

        Args:
            text (str): text to decode
        Returns:
            decodedText (str): decoded text
        """ 
        #YOUR CODE....
        pass        
        #return ....       

In [ ]:
textSample = MOText[:200]

#Create coder
huff = HuffmanCoderDecoder(symFreqs)
#Encode text
#YOUR CODE
encText = 
#decode text to check if it works
#YOUR CODE
decText = 

In [ ]:
print(encText)

In [ ]:
print(decText)

In [ ]:
print(textSample)

Вычислите, какое получилось сжатие

In [ ]:
print("Compression(%): ",len(textSample.encode('utf-8'))*8/len(encText)*100)

Поздравляем, вы сделали простой алгоритм сжатия) .... если у вас больше 100%

### Исследуем, как хорошо себя ведёт код при смене текста

Сравните написанный кодер на двух понравившихся вам текстах на разные темы (возьмите по масштабу что-то похожее на страницу МО).

1. Для чистоты эксперимента очистите оба текста от символов, которых нет в другом тексте
2. Посчитайте таблицу частот для каждого из текстов
3. Обучите кодер на тексте 1
4. Обучите кодер на тексте 2
5. Сопоставьте длину (в битах): 
   1. Текст1, закодированный кодером 1
   2. Текст2, закодированный кодером 1
   3. Текст1, закодированный кодером 2
   4. Текст2, закодированный кодером 2
6. Посчитайте KL-дивергенцию и кроссэнтропию в обе стороны между частотами кодера1 и кодера2

Прокомментируйте результат.

Не забудьте приложить к решению ваших два текстовых файла.

In [ ]:
with open("<YOURTEXT1>.txt","r", encoding="utf-8") as f:
    text1 = "\n".join(f.readlines())
with open("<YOURTEXT2>.txt","r", encoding="utf-8") as f:
    text2 = "\n".join(f.readlines())

In [ ]:
#YOUR PROCESSING
###......

In [ ]:
print( f"KL(text1 | text2)={yourResult}" )
print( f"KL(text2 | text1)={yourResult}" )
print( f"CE(text1 | text2)={yourResult}" )
print( f"CE(text2 | text1)={yourResult}" )
print( f"H(text1)={yourResult}" )
print( f"H(text2)={yourResult}" )

In [ ]:
print(f"Coding with text1: text1Len={yourResult}, text2Len={yourResult}") 
print(f"Coding with text2: text1Len={yourResult}, text2Len={yourResult}") 

In [ ]:
#ADD A CHART like https://matplotlib.org/stable/gallery/lines_bars_and_markers/barchart.html
#and draw distributions of text1 and text2
#f, ax = plt.subplots
#....

ВАШИ КОММЕНТАРИИ ЗДЕСЬ